## Assessment 1: Deep Learning

1) Answer all questions.
2) This assessment is open-book. You are allowed to refer to any references including online materials, books, notes, codes, github links, etc.
3) Copy this notebook to your google drive (click **FILE** > **save a copy in Drive**)
4) Upload the answer notebook to your github. 
5) Submit the assessment by sharing the link to your answer notebook. 





**QUESTION 1** 

One day while wandering around a clothing store at KL East Mall, you stumbled upon a pretty girl who is choosing a dress for Hari Raya. It turns out that the girl is visually impaired and had a hard time distinguishing between an abaya and a kebaya. To help people with the similar situation, you then decided to develop an AI system to identify the type of clothes using a Convolutional Neural Networks (ConvNet). In order to train the network, you decide to use the Fashion MNIST dataset which is freely available on Pytorch.


a) Given the problem, what is the most appropriate loss function to use? Justify your answer. **[5 marks]**


<span style="color:blue">
    ANSWER: Cross Entropy Loss It is is the most appropriate loss function to use. This is due to its ability to measures the performance of a classification model whose output is a probability value between 0 and 1. Cross-entropy loss increases as the predicted probability diverges from the actual label</span>

<div>
<img src="https://ml-cheatsheet.readthedocs.io/en/latest/_images/cross_entropy.png" width="300"/>
</div>

b) Create and train a ConvNet corresponding to the following CNN architecture (with a modification of the final layer to address the number of classes). Please include **[10 marks]**:

    1) The dataloader to load the train and test datasets.

    2) The model definition (either using sequential method OR pytorch class method).

    3) Define your training loop.

    4) Output the mean accuracy for the whole testing dataset.

    

<div>
<img src="https://vitalflux.com/wp-content/uploads/2021/11/VGG16-CNN-Architecture.png" width="550"/>
</div>


In [1]:
#import all independencies

import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import glob
import numpy
import random

from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchsummary import summary

In [2]:
# Deliverable 1) The dataloader to load the train and test datasets.

transform = transforms.Compose(
    [transforms.Resize((100,100)),
     transforms.ToTensor(),
     transforms.Lambda(lambda x: x.repeat(3,1,1) if x.size(0)==1 else x),
     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])


# batch_size
batch_size = 16

# datasets
trainset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [3]:
#Print/Show Data Size
train_data_size = len(trainloader.dataset)
test_data_size = len(testloader.dataset)

print(train_data_size)
print(test_data_size)

60000
10000


In [4]:
# Deliverable 2) The model definition (either using sequential method OR pytorch class method).

#1. DEFINE THE CNN 
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)     #input channel:3 for it is RGB  #output channel:6 ; how many kernel/filter you want to use #kernel size:5
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 12, 3)
        #self.batchnorm = nn.BatchNorm2d(12)

        self.conv3 = nn.Conv2d(12, 24, 3)
        self.conv4 = nn.Conv2d(24, 32, 3)
        #self.batchnorm1 = nn.BatchNorm2d(32)

        self.conv5 = nn.Conv2d(32, 40, 1)
        self.fc1 = nn.Linear(40 * 2 * 2, 120)   #size * Convolution size
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU()
  
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        #x = self.batchnorm(x)
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))     
        x = self.pool(self.relu(self.conv5(x)))
  

        #x = self.batchnorm1(x)
        #print(x.shape)
        
        x = x.view(-1, 40 * 2 * 2)
        x = self.relu(self.fc1(x))
        #nn.Dropout
        x = self.relu(self.fc2(x))
        #nn.Dropout
        x = self.fc3(x)
        return x

### PUT OUR CNN MODEL IN model

In [5]:
model = CNN() # need to instantiate the network to be used in instance method

# 2. LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 3. move the model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(32, 40, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=160, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu): ReLU()
)

In [ ]:
#Deliverable 3) Define your training loop.

import time # to calculate training time

def train_and_validate(model, loss_criterion, optimizer, epochs=25):
    '''
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param loss_criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)
  
    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    '''
    
    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in enumerate(trainloader):

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Clean existing gradients
            optimizer.zero_grad()
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs, labels)
            
            # Backpropagate the gradients
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            
            #print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

            
        # Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in enumerate(testloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)

                #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
            
        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size

        # Find average training loss and training accuracy
        avg_test_loss = valid_loss/test_data_size 
        avg_test_acc = valid_acc/test_data_size

        history.append([avg_train_loss, avg_test_loss, avg_train_acc, avg_test_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_test_loss, avg_test_acc*100, epoch_end-epoch_start))
        
        # Save if the model has best accuracy till now
        torch.save(model, 'fashinmnist'+str(epoch)+'.pt')
            
    return model, history

In [7]:
# Deliverable 4) Output the mean accuracy for the whole testing dataset.

num_epochs = 10
trained_model, history = train_and_validate(model, criterion, optimizer, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 2.3027, Accuracy: 10.0167%, 
		Validation : Loss : 2.3020, Accuracy: 10.0000%, Time: 51.6534s
Epoch: 2/10
Epoch : 001, Training: Loss: 2.1562, Accuracy: 20.9000%, 
		Validation : Loss : 0.9611, Accuracy: 64.6700%, Time: 46.7969s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.6505, Accuracy: 75.4500%, 
		Validation : Loss : 0.5473, Accuracy: 79.6300%, Time: 47.2547s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.4949, Accuracy: 81.4500%, 
		Validation : Loss : 0.4676, Accuracy: 82.9300%, Time: 45.3634s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.4350, Accuracy: 83.7767%, 
		Validation : Loss : 0.4275, Accuracy: 84.4200%, Time: 45.7895s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.3976, Accuracy: 85.1767%, 
		Validation : Loss : 0.4252, Accuracy: 84.3700%, Time: 45.3927s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.3684, Accuracy: 86.3017%, 
		Validation : Loss : 0.3882, Accuracy: 85.9100%, Time: 45.8159s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.3490, Accura

c) Replace your defined CNN in b) with a pre-trained model. Then, proceed with a transfer learning and finetune the model for the Fashion MNIST dataset. **[10 marks]**

In [8]:
###############################################
###############YOUR CODES HERE ################
###############################################

model = models.resnet18(pretrained=True)        #resnet18  #vgg19_bn #vgg11  #inception_v3 #mnasnet1_3  #alexnet

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [9]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
#######################
# DEFINE YOUR OWN MODEL
#######################
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

# Here the size of each output sample is set to 10.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, 10)                     #10 class we have                  

# 3. move the model to GPU  ####################################################   You can do number 2 first then 3
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 2. LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.00001, momentum=0.9)

In [11]:
#@title
#3) Define your training loop.

import time # to calculate training time

def train_and_validate(model, loss_criterion, optimizer, epochs=25):
    '''
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param loss_criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)
  
    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    '''
    
    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in enumerate(trainloader):

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Clean existing gradients
            optimizer.zero_grad()
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs, labels)
            
            # Backpropagate the gradients
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            
            #print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

            
        # Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in enumerate(testloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)

                #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
            
        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size

        # Find average training loss and training accuracy
        avg_test_loss = valid_loss/test_data_size 
        avg_test_acc = valid_acc/test_data_size

        history.append([avg_train_loss, avg_test_loss, avg_train_acc, avg_test_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_test_loss, avg_test_acc*100, epoch_end-epoch_start))
        
        # Save if the model has best accuracy till now
        torch.save(model, 'fashinmnist'+str(epoch)+'.pt')
            
    return model, history

In [13]:
# 4. Train the model for 10 epochs

num_epochs = 10
trained_model, history = train_and_validate(model, criterion, optimizer, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.7963, Accuracy: 77.8200%, 
		Validation : Loss : 0.5218, Accuracy: 83.0100%, Time: 105.1979s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.5232, Accuracy: 83.2567%, 
		Validation : Loss : 0.4077, Accuracy: 86.1700%, Time: 103.7765s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.4419, Accuracy: 85.3850%, 
		Validation : Loss : 0.3573, Accuracy: 87.6200%, Time: 103.7481s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.3959, Accuracy: 86.7333%, 
		Validation : Loss : 0.3288, Accuracy: 88.4700%, Time: 103.1694s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.3645, Accuracy: 87.7033%, 
		Validation : Loss : 0.3075, Accuracy: 89.1000%, Time: 104.9393s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.3405, Accuracy: 88.4400%, 
		Validation : Loss : 0.2931, Accuracy: 89.5800%, Time: 104.8038s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.3213, Accuracy: 89.1183%, 
		Validation : Loss : 0.2792, Accuracy: 90.1800%, Time: 104.8262s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.3053,

d) Using model-centric methods, propose two (2) strategies that can be used to increase the accuracy of the model on the testing dataset. **[5 marks]**


<span style="color:blue">
    Two model-centric techniques that I propose are: 1. Implementing batch normalization and dropout technique. </span>

e) Next, implement the two proposed model-centric techniques for the same problem as in the previous question. **[15 marks]**

In [14]:
###############################################
###############YOUR CODES HERE ################
###############################################
#1. DEFINE THE CNN 
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)     #input channel:3 for it is RGB  #output channel:6 ; how many kernel/filter you want to use #kernel size:5
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 12, 3)
        self.batchnorm1 = nn.BatchNorm2d(12)

        self.conv3 = nn.Conv2d(12, 24, 3)
        self.conv4 = nn.Conv2d(24, 32, 3)
        self.batchnorm2 = nn.BatchNorm2d(32)

        self.conv5 = nn.Conv2d(32, 40, 1)
        self.fc1 = nn.Linear(40 * 2 * 2, 120)   #size * Convolution size
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU()
  
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.batchnorm1(x)
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))     
        x = self.batchnorm2(x)
        x = self.pool(self.relu(self.conv5(x)))
        
        #print(x.shape)
        
        x = x.view(-1, 40 * 2 * 2)
        x = self.relu(self.fc1(x))
        nn.Dropout(0.5)
        x = self.relu(self.fc2(x))
        nn.Dropout(0.5)
        x = self.fc3(x)
        return x

In [15]:
model = CNN() # need to instantiate the network to be used in instance method

# 2. LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 3. move the model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(32, 40, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=160, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu): ReLU()
)

In [ ]:
#TRAINING LOOP HAS BEEN RUN INITIALLY

In [16]:
# 4. Train the model for 10 epochs

num_epochs = 10
trained_model, history = train_and_validate(model, criterion, optimizer, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.7465, Accuracy: 73.8917%, 
		Validation : Loss : 0.4719, Accuracy: 82.3300%, Time: 51.8917s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.4157, Accuracy: 84.5967%, 
		Validation : Loss : 0.4400, Accuracy: 83.8800%, Time: 49.5141s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.3664, Accuracy: 86.2683%, 
		Validation : Loss : 0.3794, Accuracy: 86.1600%, Time: 47.8876s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.3271, Accuracy: 87.7717%, 
		Validation : Loss : 0.3363, Accuracy: 87.6600%, Time: 47.5983s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.2975, Accuracy: 88.9167%, 
		Validation : Loss : 0.3045, Accuracy: 88.9500%, Time: 46.8590s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.2799, Accuracy: 89.6850%, 
		Validation : Loss : 0.3058, Accuracy: 88.3800%, Time: 46.2238s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.2633, Accuracy: 90.3700%, 
		Validation : Loss : 0.3032, Accuracy: 89.3300%, Time: 46.9270s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.2528, Accura

f) Do you see any accuracy improvement? Whether it is a "yes" or "no", discuss the possible reasons contributing to the accuracy improvement/ unimprovement. **[5 marks]**

<span style="color:blue">
    Your answer here </span>: YES. Both pre-trained model using resnet18 and model-centrics strategies do contribute to the improvement of model. For finetuning model already has abundance amount of pretrain ConvNet on a very large dataset (e.g. ImageNet, which contains 1.2 million images with 1000 categories). On the other hand, model-centric strategies improve the model's accuracy by normalizing the mean and standard deviation for each individual feature channel/map while dropout method approximates training a large number of neural networks with different architectures in parallel where some number of layer outputs are randomly ignored (dropped out) with probability.

g) In real applications, data-centric strategies are essential to train robust deep learning models. Give two (2) examples of such strategies and discuss how the strategies helps improving the model accuracy. **[5 marks]**

<span style="color:blue">
    Your answer here </span> 1. By the method called data augmentation which changing the features of the data such as flip/rotate. 2. Improving the quality and balancing of data such as removing blurry image and increasing/reducing dataset in class to equally balance in number.

h) Next, implement the two proposed data-centric techniques for the same problem as in the previous question. **[10 marks]**

In [45]:
###############################################
##############YOUR CODES HERE #################
###############################################
# Applying Transforms to the Data
import torchvision
import torchvision.transforms as transforms

image_transforms =  transforms.Compose(
    [transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
     transforms.RandomRotation(degrees=15),
     transforms.RandomHorizontalFlip(),
     transforms.CenterCrop(size=224),
     transforms.ToTensor(),
     transforms.Lambda(lambda x: x.repeat(3,1,1) if x.size(0)==1 else x),
     transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])

image_transforms1 = transforms.Compose([
        transforms.Resize(size=224),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3,1,1) if x.size(0)==1 else x),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

batch_size = 16

In [46]:
# datasets
trainset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=image_transforms)
testset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=image_transforms1)

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

In [44]:
###############################################
###############YOUR CODES HERE ################
###############################################
#1. THIS DEFINED CNN is the same from question e. Can directly RUN THE Training
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)     #input channel:3 for it is RGB  #output channel:6 ; how many kernel/filter you want to use #kernel size:5
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 12, 3)
        self.batchnorm1 = nn.BatchNorm2d(12)

        self.conv3 = nn.Conv2d(12, 24, 3)
        self.conv4 = nn.Conv2d(24, 32, 3)
        self.batchnorm2 = nn.BatchNorm2d(32)

        self.conv5 = nn.Conv2d(32, 40, 1)
        self.fc1 = nn.Linear(40 * 6 * 6, 120)   #size * Convolution size
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU()
  
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.batchnorm1(x)
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))     
        x = self.batchnorm2(x)
        x = self.pool(self.relu(self.conv5(x)))
        
        #print(x.shape)
        
        x = x.view(-1, 40 * 6 * 6)
        x = self.relu(self.fc1(x))
        nn.Dropout(0.5)
        x = self.relu(self.fc2(x))
        nn.Dropout(0.5)
        x = self.fc3(x)
        return x

In [47]:
model = CNN() # need to instantiate the network to be used in instance method

# 2. LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 3. move the model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(32, 40, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=1440, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu): ReLU()
)

In [25]:
##TRAINING LOOP HAS BEEN INITIALLY RUN

In [48]:
# 4. Train the model for 10 epochs

num_epochs = 10
trained_model, history = train_and_validate(model, criterion, optimizer, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.7113, Accuracy: 73.9267%, 
		Validation : Loss : 0.4779, Accuracy: 80.6300%, Time: 130.0125s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.4716, Accuracy: 82.2167%, 
		Validation : Loss : 0.3927, Accuracy: 85.7400%, Time: 124.7252s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.4225, Accuracy: 84.1533%, 
		Validation : Loss : 0.3893, Accuracy: 84.7800%, Time: 123.0821s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.3847, Accuracy: 85.4283%, 
		Validation : Loss : 0.3079, Accuracy: 88.4300%, Time: 122.8609s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.3619, Accuracy: 86.5017%, 
		Validation : Loss : 0.3174, Accuracy: 88.3500%, Time: 125.4604s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.3454, Accuracy: 87.0600%, 
		Validation : Loss : 0.3216, Accuracy: 88.0100%, Time: 125.9384s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.3289, Accuracy: 87.6983%, 
		Validation : Loss : 0.2820, Accuracy: 89.5300%, Time: 126.1306s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.3166,

**QUESTION 2** **[35 marks]**

Firstly, watch this video:

https://drive.google.com/file/d/1bsypahR7I3f_R3DXkfw_tf0BrbCHxE_O/view?usp=sharing

This video shows an example of masked face recognition where the deep learning model is able to detect and classify your face even when wearing a face mask. Using the end-to-end object detection pipeline that you have learned, develop your own masked face recognition such that the model should recognize your face even on face mask while recognize other persons as "others".

Deliverables for this question are:

- the model file. Change the name to <your_name>.pt file (e.g. hasan.pt).
- a short video (~10 secs) containing your face and your friends faces (for inference).

In [49]:
#Using roboflow

#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 12975, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 12975 (delta 81), reused 98 (delta 60), pack-reused 12844
Receiving objects: 100% (12975/12975), 13.11 MiB | 9.08 MiB/s, done.
Resolving deltas: 100% (8911/8911), done.
/content/yolov5
     |████████████████████████████████| 1.6 MB 36.1 MB/s 
     |████████████████████████████████| 145 kB 45.4 MB/s 
     |████████████████████████████████| 178 kB 67.2 MB/s 
     |████████████████████████████████| 1.1 MB 58.9 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 138 kB 67.7 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
Setup complete. Using torch 1.12.1+cu113 (Tesla T4)


In [50]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [51]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [52]:
#after following the link above, receive python code with these fields filled in
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="l54mXKeh3RAV5lnPQpEe")
project = rf.workspace("harith-aslam-va4ia").project("harith-detection")
dataset = project.version(1).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/datasets/Harith-Detection-1 in yolov5pytorch:: 100%|██████████| 262/262 [00:00<00:00, 1738.10it/s]


In [53]:
#Train our custom model

!python train.py --img 416 --batch 16 --epochs 150 --data {dataset.location}/data.yaml --weights yolov5l.pt --cache

train: weights=yolov5l.pt, cfg=, data=/content/datasets/Harith-Detection-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-180-g82bec4c Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, 